In [ ]:
using LinearAlgebra
using SparseArrays
using Plots

In [ ]:
using Revise
using LatticeQM

In [ ]:
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

# Tutorial: Hubbard mean field

## Honeycomb lattice 1

In [ ]:
lat = Geometries2D.honeycomb()
sx, sy, sz, sublA, sublB = getoperator(lat, ["SX", "SY", "SZ", "sublatticeAspin", "sublatticeBspin"])

hops = Operators.graphene(lat; mode=:spinhalf)
Operators.addzeeman!(hops, lat, r->sign(r[4]-0.5).*0.5.*[sin(0.0π),0,cos(0.0π)] )

ks = kpath(lat; num_points=200)

nothing

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(hops, ks, sz)
p1 = plot(bands; markersize=2, size=(300,200))

### Set up the interaction

In [ ]:
v = gethubbard(lat; mode=:σx, a=0.5, U=5.0) # interaction potential
ρ_init = initialguess(v, :random; lat=lat) # initial guess

nothing

In [ ]:
#Meanfield.getcappedyukawa(lat; format=:dense, spin=false, k0=0.5, U=2.3).data

### Run mean field solver

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    hops, v, ρ_init, 0.75; klin=30, iterations=800, tol=1e-5,# p_norm=Inf,
    T=0.01, β=0.7,  show_trace=true, clear_trace=true
)

nothing

### Get magnetization

In [ ]:
mA, mB = real.(Operators.magnetization(ρ_sol, [sublA,sublB], lat))
δM = mA - mB; M = mA+mB
Mabs = norm(M); δMabs = norm(δM)
dens = Operators.density(ρ_sol)
@info("Groundstate energy", ϵ_GS)
@info("Magnetization", Mabs, δMabs, M, δM)
@info("Density", dens)

### Get bands

In [ ]:
bands_mf = getbands(HMF.h, ks, sz)
bands_mf.bands .-= HMF.μ # shift chemical potential to zero
p2 = plot(bands_mf; markersize=2, size=(300,200))

### Comparison before and after

In [ ]:
plot!(p1, title="B=1.5, non-interacting")
plot!(p2, title="B=1.5, Hubbard meanfield")
plot(p1,p2, titlefont=font(8), size=(600,200))

In [ ]:
plot!(p1, title="B=1.5, non-interacting")
plot!(p2, title="B=1.5, Hubbard meanfield")
plot(p1,p2, titlefont=font(8), size=(600,200))

## Honeycomb lattice 2

In [ ]:
using LatticeQM.Meanfield

roundreal(x; digits=7) = round.(real.(x); digits=digits)

function get_gap_at_U(U=4.0; filling=0.5, init=:antiferro, T=0.01, β=0.20, show_trace=true, show_bands=false, clear_trace=false, reportmagnetization=false)
    
    # Set up system
    lat = Geometries2D.honeycomb()
    sx, sy, sz, sublA, sublB = getoperator(lat, ["sx", "sy", "sz", "sublatticeAspin", "sublatticeBspin"])
    hops = Operators.graphene(lat; mode=:spinhalf)
#     Operators.addzeeman!(hops, lat, 0.0001)

    # Set up interaction and mean field hamiltonian
    v = gethubbard(lat; mode=:σx, a=0.5, U=U) # interaction potential
    ρ_init = initialguess(v, init; lat=lat) # initial guess

    hf = hartreefock(hops, v)

    # Solve mean field
    ρ_sol, ϵ_GS, HMF, converged, error = solveselfconsistent(
        hf, ρ_init, filling; klin=30, iterations=500, tol=1e-8,# p_norm=Inf,
        T=T, β=β,  show_trace=show_trace, clear_trace=clear_trace
    )

    # Calculate the gap size
    ks  = kpath(lat; num_points=200)
    gap = Spectrum.bandgap_filling(HMF.h, ks, filling)

    if show_bands
        bands = getbands(hops, ks, sz)
        p1 = plot(bands; size=(400,300))

        bandsMF = getbands(HMF.h, ks, sz)
        bandsMF.bands .-= HMF.μ

        p2 = plot(bandsMF; size=(400,300))
        p = plot(p1,p2, size=(550,180)) #gui(...)
        display(p)
    end

    if reportmagnetization
        mA, mB = roundreal.(Operators.magnetization(ρ_sol, [sublA,sublB], lat))
        δM = mA - mB
        M = mA+mB
        Mabs = norm(M)
        δMabs = norm(δM)
        dens = Operators.density(ρ_sol)
        @info("Groundstate energy", ϵ_GS)
        @info("Magnetization", Mabs, δMabs, mA, mB, M, δM)
        @info("Density", dens)
    end

    [gap ϵ_GS]
end

In [ ]:
get_gap_at_U(3.0; filling=0.5, init=:antiferro, T=0.01, β=0.15, show_bands=true, clear_trace=true, reportmagnetization=true)

### Sweep over interaction strengths and find interaction induced band gap 

In [ ]:
using ProgressMeter

Us = range(0; stop=4.0, length=10)
data = progress_map(Us) do U
    get_gap_at_U(U; init=:antiferro, show_trace=false)
end

data = vcat(data...)
gap = data[:,1]
energies = data[:,2]

nothing

In [ ]:
scatter(Us, gap; ylim=(0,3), size=(300,200), xlabel="U", ylabel="band gap")

# Triangular lattice

## Lattice

In [ ]:
lat = Geometries2D.triangular_supercell()
sz = getoperator(lat, "sz")
sub1, sub2, sub3 = [getoperator(lat, "sublattice", i, 2) for i=1:3]

plot(lat, size=(300,150))

## Hamiltonian

In [ ]:
hops = TightBinding.addspin(Operators.nearestneighbor!(Hops(), lat), :spinhalf)

nothing

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(hops, ks, sz)
p1 = plot(bands; markersize=2, size=(300,200))

## Mean field

#### Setup

In [ ]:
v = gethubbard(lat; mode=:σx, a=0.5, U=6.0) # interaction potential
ρ_init = initialguess(v, :random; lat=lat) # initial guess
hf = hartreefock(hops, v)

nothing

#### Solver

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, error = solveselfconsistent( # run the calculation
    hf, ρ_init, 0.5; klin=30, iterations=800, tol=1e-7,# p_norm=Inf,
    T=0.01, β=0.9,  show_trace=true, clear_trace=true
)

nothing

#### Result

In [ ]:
m1,m2,m3 = real.(Operators.magnetization(ρ_sol, [sub1,sub2,sub3], lat))
m = m1+m2+m3
Mabs = norm(m)
dens = Operators.density(ρ_sol)
@info("Groundstate energy", ϵ_GS)
@info("Magnetizations", Mabs, norm(m1), norm(m2), norm(m3))
@info("Magnetization vectors", Mabs, m, m1, m2, m3)
@info("Density", dens)

In [ ]:
# Get the bands with mean-field terms
bands_mf = getbands(HMF.h, ks, sz)
bands_mf.bands .-= HMF.μ
p2 = plot(bands_mf; markersize=2, size=(600,200))

# Show the band structure side-by-side
plot!(p1, title="non-interacting")
plot!(p2, title="Hubbard meanfield")
plot(p1,p2, titlefont=font(8))

In [ ]:
using DelimitedFiles
XYZ = transpose(Structure.positions(lat))
M = transpose(hcat([m1,m2,m3]...))
mkpath("output"); writedlm("output/positions.out", XYZ)
mkpath("output"); writedlm("output/magnetization.out", M)